#### Criação do bloco e blockchain

In [11]:
# Criação do Bloco e Blockchain.
import hashlib          # Importa a biblioteca que faz o calculo do hash.
import json             # Importa o arquivo que manipula arquivos json.
from operator import index
from time import time   # Importa a função time para obter o timestamp (data e hora).

class Bloco:
    def __init__(self, index, dados, hash_anterior=""):
        # Inicializar o bloco com os parâmetros fornecidos e os atributos necessários.
        self.index = index  	            # Indice do bloco de cadeia.
        self.timestamp = time()             # Registra o momento de criação do bloco. 
        self.dados = dados                  # Dados armazenados no bloco (Ex: transações).
        self.hash_anterior = hash_anterior  # Hash do bloco anterior na cadeia.
        self.nonce = 0                      # Valor usado na prova de trabalho para alterar o hash.
        self.hash_atual = self.gerar_hash() # Gerar o hash do bloco.
    
    def gerar_hash(self):
        # Função para gerar o hash SHA-256 do bloco.
        conteudo_bloco = json.dumps({
            'index': self.index,
            'timestamp': self.timestamp,
            'dados': self.dados,
            'hash_anterior': self.hash_anterior,
            'nonce': self.nonce
        }, sort_keys=True).encode()         # Converte os dados do bloco em uma string JSON ordenada
        return hashlib.sha256(conteudo_bloco).hexdigest()  # Calcula o hash e retorna como uma string hexadecimal.
    
    def prova_de_trabalho(self, dificuldade):
        # Realiza a prova de trabalho ajustando o nonce até que o hash comece com 'dificuldade' zeros.
        while self.hash_atual[:dificuldade] != '0' * dificuldade:   #
            self.nonce += 1                                         # Incrementa o nonce para alterar o hash.
            self.hash_atual = self.gerar_hash()                     # Recalcula o hash com o novo nonce.
    
class Blockchain:
    def __init__(self):
        # Inicia a cadeia de blocos com o bloco Gênesis (primeiro bloco)e defini a dificuldade da prova de trabalho.
        self.cadeia = [self.criar_bloco_genesis()]                  # Lista que armazena os Blocos na cadeia.
        self.dificuldade = 2                                        # Define o numero de zeros necessários no hash da prova de trabalho.
        
    def criar_bloco_genesis(self):
        # Cria um bloco inicial com dados fixos.
        return Bloco(0, "Bloco Gênesis", "0")                       # Cria o bloco genesis.
    
    def obter_ultimo_bloco(self):
        # Retorna o último bloco da cadeia.
        if len(self.cadeia) < 1:
            return None
        return self.cadeia[-1] 
        
    def adicionar_bloco(self, novo_bloco):
        # Adiciona um novo bloco a cadeia.
        novo_bloco.hash_anterior = self.obter_ultimo_bloco().hash_atual     # Define o hash do bloco anterior.
        novo_bloco.prova_de_trabalho(self.dificuldade)                      # Realiza a prova de trabalho para alterar o hash. 
        self.cadeia.append(novo_bloco)                                      # Adiciona o novo bloco à cadeia.
    
    def validar_cadeia(self):
        # Valida a integridade da cadeia de blocos.
        for i in range(1, len(self.cadeia)):
            bloco_atual = self.cadeia[i]
            bloco_anterior = self.cadeia[i-1]
            
            # Verifica se o hash do bloco atual é válido.
            if bloco_atual.hash_atual!= bloco_atual.gerar_hash():
                return False
            
            # Verifica se o hash do bloco anterior é válido.
            if bloco_atual.hash_anterior!= bloco_anterior.hash_atual:
                return False
        
        return True                                                         # Retorna verdadeiro se a cadeia for valida.
    
    def salvar_em_json(self, nome_arquivo='blockchain.json'):
        # Salva a cadeia de blocos em um arquivo JSON.
        with open(nome_arquivo, 'w') as arquivo:
            json.dump([bloco.__dict__ for bloco in self.cadeia], arquivo, indent=4)
        
# Função para coletar dados de exames médicos. 
def coletar_dados_exame():
    # Implementar a função para coletar dados de exames médicos.
    glicose = float(input('Digite o valor da glicose: '))
    return {
        'glicose': glicose
    }                               # Retorna um dicionario com os dados coletados. 


### Instanciar a blockchain

In [12]:
blockchain = Blockchain()

#### Adicionar um bloco

In [16]:
# Coletar os dados de exame do usuario
dados_exame = coletar_dados_exame()

# Criar um novo bloco com os dados coletados
novo_bloco = Bloco(blockchain.obter_ultimo_bloco().index + 1 , dados_exame)
# Define o indice do novo bloco como indice do ultimo bloco + 1
# Associa os dados coletados ao bloco criado

# Adiciona o novo bloco a blockchain 
blockchain.adicionar_bloco(novo_bloco)

# Verificar a integridade da blockchain
if blockchain.validar_cadeia():
    blockchain.salvar_em_json()
    print("Blockchain valida!")
else:
    print("Blockchain Invalida!")

Blockchain valida!


#### Exibir o conteudo do arquivo JSON

In [19]:
def print_json_content(filename):
    #tenta abrir e processar o arquivo json
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
            print(json.dumps(data, indent=4))

    except FileNotFoundError:
        print(f"Erro: Arquivo {filename} não encontrado")
    except json.JSOMDecodeError:
        print(f"Erro: Arquivo {filename} não é um json valido")
print_json_content('blockchain.json')

[
    {
        "index": 0,
        "timestamp": 1733785894.0300276,
        "dados": "Bloco G\u00eanesis",
        "hash_anterior": "0",
        "nonce": 0,
        "hash_atual": "5aecdafd0b38bcb54d0aaeb0214e122e60b8a23f9e8356a4c5f9d61e59fe6bad"
    },
    {
        "index": 1,
        "timestamp": 1733785932.4006116,
        "dados": {
            "glicose": 450.0
        },
        "hash_anterior": "5aecdafd0b38bcb54d0aaeb0214e122e60b8a23f9e8356a4c5f9d61e59fe6bad",
        "nonce": 407,
        "hash_atual": "00e95df411f9b004f6963a6fc257bc0f43172474a2557df92103bffa56bdef61"
    },
    {
        "index": 2,
        "timestamp": 1733786181.5748737,
        "dados": {
            "glicose": 450.0
        },
        "hash_anterior": "00e95df411f9b004f6963a6fc257bc0f43172474a2557df92103bffa56bdef61",
        "nonce": 124,
        "hash_atual": "0030d198486b6bbefb993bdb232e32821e36d885d155d6c84f9706ed1084f6fb"
    },
    {
        "index": 3,
        "timestamp": 1733786211.6874392,
     